In [120]:
import os
from gensim.models import Word2Vec
from url_sequences.sequence_manager import *
from url_sequences.sequence_plotter import *
import numpy as np
from sklearn.manifold import TSNE
import hdbscan
import seaborn as sns
import sklearn

In [93]:
def get_colors(n):
    # 1.arancione, 2.bianco, 3.giallo, 4.azzurro, 5.verde, 6.blu, 7.fucsia, 8.viola
    colors = ["#FF8F00", "#FFFFFF", "#FFFF00", "#00E5FF", "#76FF03", "#2979FF", "#F50057", "#9C27B0"]
    c = ""
    if n < 0:
        c = colors[7]
    else:
        c = colors[n]
    return c

# WORD2VEC MODEL

In [4]:
path = os.getcwd() + "/dataset/cs.illinois.edu/RandomWalkLists-depth.100000.seqLen.10/"
rwwl_map_path = path + "sequencesMapUrl.txt"
rwwl_seq_path = path + "sequencesIDs.txt"

In [97]:
vocab_sequences = get_seq(rwwl_seq_path, 1)
train_sequences = get_seq(rwwl_seq_path, 1)

In [98]:
model = Word2Vec(min_count=1)
model.build_vocab(vocab_sequences)
model.train(train_sequences)

890323

In [101]:
model.vocab

{'u_0': <gensim.models.word2vec.Vocab at 0x7fa2272f7890>,
 'u_1': <gensim.models.word2vec.Vocab at 0x7fa2272f7850>,
 'u_10': <gensim.models.word2vec.Vocab at 0x7fa217d1c290>,
 'u_100': <gensim.models.word2vec.Vocab at 0x7fa227306790>,
 'u_101': <gensim.models.word2vec.Vocab at 0x7fa2273067d0>,
 'u_102': <gensim.models.word2vec.Vocab at 0x7fa227306810>,
 'u_103': <gensim.models.word2vec.Vocab at 0x7fa227306850>,
 'u_104': <gensim.models.word2vec.Vocab at 0x7fa227306690>,
 'u_105': <gensim.models.word2vec.Vocab at 0x7fa2273066d0>,
 'u_106': <gensim.models.word2vec.Vocab at 0x7fa227306710>,
 'u_107': <gensim.models.word2vec.Vocab at 0x7fa227306750>,
 'u_108': <gensim.models.word2vec.Vocab at 0x7fa217d1d690>,
 'u_109': <gensim.models.word2vec.Vocab at 0x7fa217d1d750>,
 'u_11': <gensim.models.word2vec.Vocab at 0x7fa217d1c250>,
 'u_110': <gensim.models.word2vec.Vocab at 0x7fa217d34610>,
 'u_111': <gensim.models.word2vec.Vocab at 0x7fa217d345d0>,
 'u_112': <gensim.models.word2vec.Vocab at 0x7

# TSNE 2-DIM

In [43]:
seq_map = get_seq_map(rwwl_map_path)

# 100-dim vecs
wordvecs = []
[wordvecs.append(model[key]) for key in seq_map]

# long-url labels
word_labels = []
[word_labels.append(seq_map[key]) for key in seq_map]

high_dim_array = np.array(wordvecs, dtype="float64")
two_dim = TSNE(n_components=2)
np.set_printoptions(suppress=True)

# 2-dim vecs
two_dim_wordvecs = two_dim.fit_transform(high_dim_array)

In [103]:
seq_map["u_140"]
word_labels[0]

model["u_140"]
wordvecs[0]

high_dim_array[0]
two_dim_wordvecs[0]

array([ -0.22848746, -20.07811845])

In [45]:
print len(two_dim_wordvecs)
print two_dim_wordvecs
type(high_dim_array[0, 0])

908
[[ -0.22848746 -20.07811845]
 [ 14.46741859  16.4208896 ]
 [ -0.44871556 -19.72042349]
 ..., 
 [ -7.9146981    2.10990458]
 [ 20.1470588  -14.47540428]
 [ 18.76750944  -8.06187561]]


numpy.float64

# CLUSTERING DBSCAN

In [9]:
from sklearn.cluster import DBSCAN

In [25]:
dbscan = DBSCAN(eps=0.4, min_samples=10)
dbscan.fit(two_dim_wordvecs)

colors = []
# [colors.append(get_colors(l)) for l in dbscan.labels_]

print dbscan.labels_

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1

# SCATTER PLOT

In [11]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [12]:
def sc_plot(two_dim_vecs, word_labes=None, colors="#FFFF00"):
    py.sign_in('chrispol', 'yvg1d17o5q')
    
    x_coord = two_dim_vecs[:, 0]
    y_coord = two_dim_vecs[:, 1]
    
    trace = go.Scattergl(
        x = x_coord, #
        y = y_coord, #
        mode = 'markers',
        text = word_labels, #
        marker = dict(
            color = colors,
            line = dict(width = 1)
        )
    )

    data = [trace]
    return data

data = sc_plot(two_dim_wordvecs, word_labels, colors)
py.iplot(data, filename='Word Vectors - Scatter plot')

# HDBSCAN

In [208]:
hdbscan_clusterer = hdbscan.HDBSCAN(min_cluster_size=10)
hdbscan_labels = hdbscan_clusterer.fit_predict(high_dim_array)

In [209]:
print set(hdbscan_labels)

set([0, 1, 2, 3, 4, 5, 6, 7, -1])


In [51]:
def plot_clusters(data, algorithm, args, kwds):
    start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    end_time = time.time()
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    plt.scatter(data.T[0], data.T[1], c=colors, **plot_kwds)
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    plt.title('Clusters found by {}'.format(str(algorithm.__name__)), fontsize=24)
    plt.text(-0.5, 0.7, 'Clustering took {:.2f} s'.format(end_time - start_time), fontsize=14)

In [62]:
palette = sns.color_palette('deep', np.unique(cluster_labels).max() + 1)
colorss = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in cluster_labels]
type (palette)


seaborn.palettes._ColorPalette

In [206]:
colors = []
[colors.append(get_colors(l)) for l in hdbscan_labels]

data = sc_plot(two_dim_wordvecs, word_labels, colors)
py.iplot(data, filename='Word Vectors - Scatter plot HDBSCAN')


In [67]:
from sklearn.cluster import KMeans

In [141]:
model = KMeans(n_clusters=8)
k = model.fit(wordvecs)


In [142]:
colors = []
[colors.append(get_colors(l)) for l in k.labels_]

data = sc_plot(two_dim_wordvecs, word_labels, colors)
py.iplot(data, filename='Word Vectors - Scatter plot KMEANS')

In [81]:
csv_path = path + "sequencesMapUrl-manually-clusterized.txt"
tuple_list = get_sequence_tuple_list(csv_path)
print tuple_list[0]

('https://cs.illinois.edu/', 'u_0', '7')


In [92]:
import csv
real_clust_map = {}
with open(csv_path, 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        real_clust_map[row[1].strip()] = row[2].strip()
        
# print real_clust_map

In [112]:
from_kmeans_to_real = {}
url_to_cluster_map = {}
for t in tuple_list:
    url_to_cluster_map[t[1]] = int(t[2])
    
url_to_cluster_map["u_480"]


4

In [115]:
colors = []
for key in seq_map:
    colors.append(get_colors(url_to_cluster_map[key]))
# [colors.append(get_colors(l)) for l in k.labels_]
data = sc_plot(two_dim_wordvecs, word_labels, colors)
py.iplot(data, filename='Word Vectors - Scatter plot MANUALLY')

In [165]:
i = {}
for t in tuple_list:
    i[t[1]] = int(t[2])
for key in seq_map:
    print seq_map[key], "---", i[key]

 https://cs.illinois.edu/about-us/about-siebel-center/ --- 0
https://cs.illinois.edu/courses/profile/CS420/ --- 1
https://cs.illinois.edu/about-us/resources/ --- 0
https://cs.illinois.edu/news/gropp-vaidya-named-ieee-fellows/ --- 4
https://cs.illinois.edu/news/illinois-cct-featured-ieee-computer-magazine-cover-story/ --- 4
https://cs.illinois.edu/prospective-students/graduate-students/ms-phd-programs/graduate-fellowships-awards/national-defense-/ --- 5
https://cs.illinois.edu/directory/profile/nikita/ --- 3
https://cs.illinois.edu/news/phd-student-awarded-1-10-nvidia-graduate-fellowships/ --- 4
https://cs.illinois.edu/prospective-students/graduate-students/additional-application-materials-international-students-only/ --- 5
https://cs.illinois.edu/courses/profile/CS498IDS/ --- 1
https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/susan-silver-and-ross-erle/ --- 5
https://cs.illinois.edu/news/klara-nahrstedt-wins-prestigious-humboldt-research

In [169]:
real_clust = []
#for t in tuple_list:
    #real_clust.append(int(t[2]))

for key in i:
    real_clust.append(i[key])
    
real_clust = np.array(real_clust)
print type(real_clust[0]), type(k.labels_[0])

print "precision: ", sklearn.metrics.precision_score(real_clust, k.labels_)
print "recall: ", sklearn.metrics.recall_score(real_clust, k.labels_)

<type 'numpy.int64'> <type 'numpy.int32'>
precision:  0.19703082043
recall:  0.0803964757709


/home/chris/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1203: DeprecationWarning:

The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".

/home/chris/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1304: DeprecationWarning:

The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".



In [167]:
for key in seq_map:
    print seq_map[key], i[key]

https://cs.illinois.edu/about-us/about-siebel-center/ 0
https://cs.illinois.edu/courses/profile/CS420/ 1
https://cs.illinois.edu/about-us/resources/ 0
https://cs.illinois.edu/news/gropp-vaidya-named-ieee-fellows/ 4
https://cs.illinois.edu/news/illinois-cct-featured-ieee-computer-magazine-cover-story/ 4
https://cs.illinois.edu/prospective-students/graduate-students/ms-phd-programs/graduate-fellowships-awards/national-defense-/ 5
https://cs.illinois.edu/directory/profile/nikita/ 3
https://cs.illinois.edu/news/phd-student-awarded-1-10-nvidia-graduate-fellowships/ 4
https://cs.illinois.edu/prospective-students/graduate-students/additional-application-materials-international-students-only/ 5
https://cs.illinois.edu/courses/profile/CS498IDS/ 1
https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/susan-silver-and-ross-erle/ 5
https://cs.illinois.edu/news/klara-nahrstedt-wins-prestigious-humboldt-research-award/ 4
https://cs.illinois.edu/news/gropp-

In [215]:
arr = [0, 0, 0, 0, 0, 0, 0, 0]
matr = np.zeros((len(set(real_clust)),len(set(hdbscan_labels))))

distinct_clusters = set(real_clust)
#print distinct_clusters

for i in distinct_clusters:
    for n in range(len(real_clust)):
        if real_clust[n] == i:
            matr[i, hdbscan_labels[n]] = matr[i, hdbscan_labels[n]] + 1
            # print n

#for n in range(len(real_clust)):
#    if real_clust[n] == 1:
#        matr[0, k.labels_[n]] = matr[0, k.labels_[n]] + 1
#        print n

# real_clust x clust_labels

print set(real_clust)
print set(hdbscan_labels)
print matr

908
set([0, 1, 2, 3, 4, 5, 6, 7])
set([0, 1, 2, 3, 4, 5, 6, 7, -1])
[[   0.    2.    0.    1.    0.   19.    2.    2.   13.]
 [   1.    2.    1.    3.    9.   46.    7.    9.   48.]
 [   2.    0.    0.    6.    3.   12.    3.    1.   16.]
 [   4.    3.    1.    2.    7.  118.   35.   23.   86.]
 [   7.    1.    4.    4.    6.  168.   10.    6.   66.]
 [   1.    1.    3.    3.    1.   54.    5.    4.   38.]
 [   0.    0.    0.    0.    0.    5.    1.    1.    6.]
 [   0.    1.    2.    0.    1.   10.    0.    0.   12.]]


In [205]:
j = 0
for i in k.labels_:
    if i == 0:
        j = j + 1
print j

466
